In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *

%matplotlib inline

In [2]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [3]:
v = t.T[[
    'R00200', 'R00268', 'R00341', 'R00342', 'R00351',
    'R00405', 'R00658', 'R00703', 'R01015', 'R01049',
    'R01056', 'R01061', 'R01070', 'R01082', 'R01196', 
    'R01197', 'R01325', 'R01512', 'R01518', 'R01528',
    'R01529', 'R01641', 'R01786', 'R01830', 'R01899',
    'R01900', 'R02035', 'R02164', 'R02740', 'R04779'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00014'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00025'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

# v['C00279'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [4]:
# annmonia output
# Glutamate → Oxaloacetate
#                    → Glutamate
# Glutamate → 

In [5]:
df = v.T
df['Glucose_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose_input']['C00267'] = 1

df['Oxygen_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen_input']['C00007'] = 1

df['CO2_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2_output']['C00011'] = -1

df['Lactate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate_output']['C00186'] = -1

df['Succinate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Succinate_output']['C00042'] = -1

df['Malate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Malate_output']['C00149'] = -1

df['Fumarate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Fumarate_output']['C00122'] = -1

df['Acetate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate_output']['C00033'] = -1

df['Glutamate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glutamate_output']['C00025'] = -1

df['Glutamate_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glutamate_input']['C00025'] = 1

df['Ammonia_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Ammonia_output']['C00014'] = -1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['ADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['ADP_Exchange']['C00002'] = -1
df['ADP_Exchange']['C00008'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00835']['C00092'] = -1
df['R00835']['C00006'] = -1
df['R00835']['C01236'] = 1
df['R00835']['C00005'] = 1
df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

df['R00243'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00243']['C00025'] = -1
df['R00243']['C00003'] = -1
df['R00243']['C00001'] = -1

df['R00243']['C00026'] = 1
df['R00243']['C00014'] = 1
df['R00243']['C00004'] = 1
df['R00243']['C00080'] = 1

# Oxophos
df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [6]:
BIOMASS = pd.read_csv('../csv/amino_acid.csv',index_col=0)
BIOMASS = BIOMASS[BIOMASS.index != 'NH3']
df_Biomass =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in BIOMASS.index:
    df_Biomass[BIOMASS.T[i]['C_num']] = BIOMASS.T[i]['flux']
    
df_Biomass['C00668'] = -0.27519
df_Biomass['C00119'] = (-0.039036) + (-0.036117) + (-0.053446) + (-0.039036)

In [7]:
df_new = pd.concat([df_Biomass.T, df], axis=1)
df_new = df_new.rename(columns={0: 'BIOMASS'})
df_new = df_new.drop([
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603', 
#     'C00231'
])

In [8]:
x = []
for i in df_new.columns:
    if (
            (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or (i=='R02035') or
            (i=='CO2') or (i == 'Lactate') or (i == 'Acetate') or (i == 'Succinate') or (i == 'Malate') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or
            (i=='Glutamate_output')
    ) :
        x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

    elif (i=='Glucose_input'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))

    elif (i=='Glutamate_input'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='Fumarate_output'):
        x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))    
        
    elif (i=='Oxygen_input'):
        x.append(LpVariable(i, lowBound=0, upBound=1, cat='Continuous'))

    elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
        x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

    else:
        x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

In [9]:
m = LpProblem(sense=LpMaximize)
m += x[0]

for i in np.dot(df_new, x):
    m += i == 0

In [10]:
status = m.solve()
print(LpStatus[status])

Optimal


In [11]:
y = []
for i in x:
    y.append(i.value())
    
f=pd.DataFrame(y, index=x)
f[f[0] != 0]

,0
BIOMASS,508.613960
R00268,3276.016500
R00341,662.612330
R00342,4442.812700
R00351,-3276.016500
R00405,-2621.431700
R00658,-662.612330
R00703,257.170480
R01015,68.385977
R01049,85.261502


In [12]:
result = df_new*y
for i in result:
     if sum(result[i] == 0) == result.shape[0]:
            result=result.drop(i,axis=1)

In [13]:
def search(x):
    if x[0] == 'C':
        return pd.DataFrame(result.T[result.T[x] != 0 ][x])
        
    else:
        return pd.DataFrame(result[result[x] != 0 ][x])

In [14]:
search('C00022')

,C00022
BIOMASS,-257.170477
R00703,257.170480
